In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./make.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.atp-autoteile.de/de/product/vehicles/' + input_.loc[a, 'Product_Id'] + '/' + input_.loc[a, 'Make_Code'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_td = html.xpath('//td')
                
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for td in list_td:
                    model = td.xpath('./span[1]/text()')[0].strip() + ' ' + td.xpath('./span[2]/text()')[0][:-1].strip() if td.xpath('./span[2]/text()')[0].endswith(',') else td.xpath('./span[2]/text()')[0].strip()
                    engine = td.xpath('./span[3]/text()')[0][:-1].strip() if td.xpath('./span[3]/text()')[0].endswith(',') else td.xpath('./span[3]/text()')[0].strip()
                    year = td.xpath('./span[4]/text()')[0].strip()

                    if model not in dict_vehicle:
                        if '-' in year:
                            dict_vehicle[model] = {engine: [int(year.split('-')[0].split('/')[1].strip()), int(year.split('-')[1].split('/')[1][:-1].strip())]}
                        else:
                            dict_vehicle[model] = {engine: [int(year.split('/')[1][:-1].strip())]}
                    else:
                        if engine not in dict_vehicle[model]:
                            if '-' in year:
                                dict_vehicle[model][engine] = [int(year.split('-')[0].split('/')[1].strip()), int(year.split('-')[1].split('/')[1][:-1].strip())]
                            else:
                                dict_vehicle[model][engine] = [int(year.split('/')[1][:-1].strip())]
                        else:
                            if '-' in year:
                                dict_vehicle[model][engine].append(int(year.split('-')[0].split('/')[1].strip()))
                                dict_vehicle[model][engine].append(int(year.split('-')[1].split('/')[1][:-1].strip()))
                            else:
                                dict_vehicle[model][engine].append(int(year.split('/')[1][:-1].strip()))
     
                # = = = = = = = = = = = = = = =
        
                list_vehicle = []
                for model in dict_vehicle:
                    list_year = []
                    for engine in dict_vehicle[model]:
                        list_year += dict_vehicle[model][engine]
                    list_year = sorted(list(set(list_year)))

                    year_min = min(list_year)
                    year_max = max(list_year)
                    if year_min == year_max:
                        list_vehicle.append(input_.loc[a, 'Make'] + ' ' + model + ' ' + str(year_min))
                    else:
                        list_vehicle.append(input_.loc[a, 'Make'] + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                list_vehicle.sort()

                vehicle_1 = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
            
                list_vehicle = []
                for model in dict_vehicle:
                    for engine in dict_vehicle[model]:
                        year_min = min(dict_vehicle[model][engine])
                        year_max = max(dict_vehicle[model][engine])
                        if year_min == year_max:
                            list_vehicle.append(input_.loc[a, 'Make'] + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                        else:
                            list_vehicle.append(input_.loc[a, 'Make'] + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                list_vehicle.sort()

                vehicle_2 = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                         'Product_Id': input_.loc[a, 'Product_Id'],
                                         'Vehicle_1': vehicle_1,
                                         'Vehicle_2': vehicle_2}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'序号': int(input_.loc[a, '序号']),
                                     'Product_Id': input_.loc[a, 'Product_Id'],
                                     'Make': input_.loc[a, 'Make'],
                                     'Make_Code': input_.loc[a, 'Make_Code']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Product_Id'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Vehicle_1', 'Vehicle_2'], ascending=[True, True, True]).reset_index(drop=True)
output_correct.to_excel('./model-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Make_Code'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./model_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：277

[ok] - 1093509
[尝试次数：1] - [剩余数量：257] - [当前时间：14:34:10]

[ok] - 1093509
[尝试次数：1] - [剩余数量：256] - [当前时间：14:34:10]

[ok] - 4234518
[尝试次数：1] - [剩余数量：255] - [当前时间：14:34:10]

[ok] - 1144742
[尝试次数：1] - [剩余数量：254] - [当前时间：14:34:10]

[ok] - 1127436
[尝试次数：1] - [剩余数量：253] - [当前时间：14:34:10]

[ok] - 4098330
[尝试次数：1] - [剩余数量：252] - [当前时间：14:34:10]

[ok] - 1144742
[尝试次数：1] - [剩余数量：251] - [当前时间：14:34:10]

[ok] - 1144742
[尝试次数：1] - [剩余数量：250] - [当前时间：14:34:11]

[ok] - 1093509
[尝试次数：1] - [剩余数量：249] - [当前时间：14:34:11]

[ok] - 3648954
[尝试次数：1] - [剩余数量：248] - [当前时间：14:34:11]

[ok] - 3648954
[尝试次数：1] - [剩余数量：247] - [当前时间：14:34:11]

[ok] - 4316658
[尝试次数：1] - [剩余数量：246] - [当前时间：14:34:11]

[ok] - 988398
[尝试次数：1] - [剩余数量：245] - [当前时间：14:34:11]

[ok] - 4098330
[尝试次数：1] - [剩余数量：244] - [当前时间：14:34:11]

[ok] - 4234508
[尝试次数：1] - [剩余数量：243] - [当前时间：14:34:11]

[ok] - 4098330
[尝试次数：1] - [剩余数量：242] - [当前时间：14:34:11]

[ok] - 4316295
[尝试次数：1] - [剩余数量：241] - [当前时间：14:34:11]

[ok] - 3642704
[尝试次数：1] - [剩余数量：240] - [

[ok] - 3644700
[尝试次数：1] - [剩余数量：109] - [当前时间：14:34:33]

[ok] - 56659
[尝试次数：5] - [剩余数量：108] - [当前时间：14:34:33]

[ok] - 4305427
[尝试次数：4] - [剩余数量：107] - [当前时间：14:34:33]

[ok] - 4407355
[尝试次数：1] - [剩余数量：106] - [当前时间：14:34:33]

[ok] - 986633
[尝试次数：4] - [剩余数量：105] - [当前时间：14:34:34]

[ok] - 1070228
[尝试次数：1] - [剩余数量：104] - [当前时间：14:34:34]

[ok] - 4329100
[尝试次数：1] - [剩余数量：103] - [当前时间：14:34:34]

[ok] - 4118034
[尝试次数：1] - [剩余数量：102] - [当前时间：14:34:35]

[ok] - 4092330
[尝试次数：1] - [剩余数量：101] - [当前时间：14:34:35]

[ok] - 4329100
[尝试次数：1] - [剩余数量：100] - [当前时间：14:34:36]

[ok] - 4307160
[尝试次数：1] - [剩余数量：99] - [当前时间：14:34:36]

[ok] - 4405831
[尝试次数：1] - [剩余数量：98] - [当前时间：14:34:36]

[ok] - 988458
[尝试次数：1] - [剩余数量：97] - [当前时间：14:34:37]

[ok] - 4407355
[尝试次数：1] - [剩余数量：96] - [当前时间：14:34:37]

[ok] - 4312390
[尝试次数：1] - [剩余数量：95] - [当前时间：14:34:37]

[ok] - 1070229
[尝试次数：4] - [剩余数量：94] - [当前时间：14:34:37]

[ok] - 4091725
[尝试次数：1] - [剩余数量：93] - [当前时间：14:34:38]

[ok] - 44596
[尝试次数：1] - [剩余数量：92] - [当前时间：14:34:38]

[ok] -

In [2]:
df = output_correct[['Product_Id', 'Vehicle_1', 'Vehicle_2']]
df = df.groupby('Product_Id', as_index=False).agg({'Vehicle_1': '\n'.join, 'Vehicle_2': '\n'.join})
df.to_excel('./model.xlsx', index=False)

df

,Product_Id,Vehicle_1,Vehicle_2
0,1000392,"VW TRANSPORTER T4 Bus (70B, 70C, 7DB, 7DK, 70J...","VW TRANSPORTER T4 Bus (70B, 70C, 7DB, 7DK, 70J..."
1,1020670,"Audi A3 (8V1, 8VK) 1.8 TFSI 2012-2016\nAudi A3...","Audi A3 (8V1, 8VK) 1.8 TFSI 2012-2016 132kW / ..."
2,103013,"Mercedes-Benz SPRINTER 4,6-t Pritsche/Fahrgest...","Mercedes-Benz SPRINTER 4,6-t Pritsche/Fahrgest..."
3,1040042,"Audi A4 (8D2, B5) 1.8 T 1995-2000\nAudi A4 (8D...","Audi A4 (8D2, B5) 1.8 T 1995-2000 110kW / 150P..."
4,1051673,"Mercedes-Benz SPRINTER 3,5-t Bus (906) 310 CDI...","Mercedes-Benz SPRINTER 3,5-t Bus (906) 310 CDI..."
...,...,...,...
145,988695,Opel SIGNUM CC (Z03) 1.8 2003-2008\nOpel SIGNU...,Opel SIGNUM CC (Z03) 1.8 2003-2005 90kW / 122P...
146,988750,Opel ASTRA H (A04) 1.2 2005-2010\nOpel ASTRA H...,Opel ASTRA H (A04) 1.2 2005-2010 59kW / 80PS\n...
147,988801,"Audi A4 (8D2, B5) 1.9 TDI 2000\nAudi A4 (8EC, ...","Audi A4 (8D2, B5) 1.9 TDI 2000 85kW / 116PS\nA..."
148,989080,BMW 3 (E46) 316 i 1998-2005\nBMW 3 (E46) 318 d...,BMW 3 (E46) 316 i 1998-2005 77kW / 105PS\nBMW ...
